In [1]:
import pandas as pd
import numpy as np
import xarray as xr
import geopandas as gpd 

%load_ext autoreload
%autoreload 2

#### Read ECMWF seasonal forecasts

In [11]:
month = "may"
country = "USA" # USA or BRAZIL

In [12]:
filtered_coords = pd.read_csv("../data/ecmwf_coords_in_us.csv")
# read hindcasts
list_hindcasts = []
for filename in ["{}_2003_09".format(month), "{}_2010_16".format(month), "{}_2017_23".format(month)]:
    print("reading {}".format(filename))
    df = xr.open_dataset("../data/{}/ecmwf_grib/".format(country) + filename.lower() + ".grib")
    print("ensembles mean") # Note: This is subject to change for probabilistic forecasting
    df = df.mean("number")
    print("to dataframe")
    df = df.to_dataframe().dropna().reset_index()
    print("filter coordinates to continental US")
    df = df.merge(filtered_coords[["latitude", "longitude"]], on=["latitude", "longitude"], how="right")
    print("convert to datetime")
    df["valid_time"] = pd.to_datetime(df["valid_time"])
    df["time"] = pd.to_datetime(df["time"])
    list_hindcasts.append(df)

ecmwf = pd.concat(list_hindcasts, ignore_index=True)

reading may_2003_09
ensembles mean
to dataframe
filter coordinates to continental US
convert to datetime
reading may_2010_16
ensembles mean
to dataframe
filter coordinates to continental US
convert to datetime
reading may_2017_23
ensembles mean
to dataframe
filter coordinates to continental US
convert to datetime


## Export to csv

In [13]:
ecmwf.to_csv("../data/ecmwf_csv/ecmwf_{}.csv".format(month), index=False)